In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import Bio
from Bio import SeqIO
from pandas import Series
import argparse
import re

In [3]:
chrlist = [''.join(['chr', str(i)]) for i in list(range(1, 23)) + ['X', 'Y']]

fa = dict()
for record in SeqIO.parse('/Users/yzli/weilab/project/reference/hg38_genome.fa','fasta'):
    if record.id in chrlist:
        fa[record.id] = record

In [4]:
#forward strand sgRNA

In [5]:
nosp = pd.read_csv('../data/ser/nosp_s_genpos.txt',sep='\t')
sp1_23 = pd.read_csv('../data/ser/sp1_23_s_genpos.txt',sep='\t')
sp12_3 = pd.read_csv('../data/ser/sp12_3_s_genpos.txt',sep='\t')

In [6]:
sp1_23 = sp1_23[['chr','feature','strand','gene_name','transcript_id',
               'pro_id','base1','base2','base3','aa_genome_seq','aa_position']]
sp12_3 = sp12_3[['chr','feature','strand','gene_name','transcript_id',
               'pro_id','base1','base2','base3','aa_genome_seq','aa_position']]

In [7]:
nosp.loc[:,'type'] = 'nosp'
sp1_23.loc[:,'type'] = 'sp1_23'
sp12_3.loc[:,'type'] = 'sp12_3'

In [8]:
genpos_all = nosp.append(sp1_23).append(sp12_3)

In [9]:
genpos_all.shape

(501263, 12)

In [10]:
genpos_f = genpos_all[genpos_all['strand'] == '+']
genpos_r = genpos_all[genpos_all['strand'] == '-']

In [11]:
genpos_f.loc[:,'reg_from'] = genpos_f['base1'] + 14
genpos_f.loc[:,'reg_to'] = genpos_f['base1'] + 20

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
genpos_f.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to
0,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400307,18400308,18400309,AGC,102,nosp,18400321,18400327
1,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400382,18400383,18400384,AGC,127,nosp,18400396,18400402
2,chr1,CDS,+,ACADM,NM_000016,ACADM_HUMAN,75728410,75728411,75728412,AGT,14,nosp,75728424,75728430
3,chr1,CDS,+,ACADM,NM_000016,ACADM_HUMAN,75728482,75728483,75728484,AGT,38,nosp,75728496,75728502
4,chr1,CDS,+,ACADM,NM_000016,ACADM_HUMAN,75733566,75733567,75733568,AGT,109,nosp,75733580,75733586


In [13]:
genpos_f.loc[:,'reg_seq'] = genpos_f.apply(
            lambda x: str(fa[x['chr']].seq[x['reg_from'] - 1: x['reg_to']]).upper(),
            axis=1)

In [14]:
genpos_f.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq
0,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400307,18400308,18400309,AGC,102,nosp,18400321,18400327,TCACCTT
1,chr8,CDS,+,NAT2,NM_000015,ARY2_HUMAN,18400382,18400383,18400384,AGC,127,nosp,18400396,18400402,GTGGCAG
2,chr1,CDS,+,ACADM,NM_000016,ACADM_HUMAN,75728410,75728411,75728412,AGT,14,nosp,75728424,75728430,TCATTGG
3,chr1,CDS,+,ACADM,NM_000016,ACADM_HUMAN,75728482,75728483,75728484,AGT,38,nosp,75728496,75728502,TTCGGTT
4,chr1,CDS,+,ACADM,NM_000016,ACADM_HUMAN,75733566,75733567,75733568,AGT,109,nosp,75733580,75733586,TTATGGA


In [15]:
genpos_f.shape

(258139, 15)

In [16]:
pam_f = []
for index, data in genpos_f.iterrows():
    for i in re.finditer('(?=GG)',data['reg_seq']):
        NGG_pos = i.span()[1]
        pam_f.append([data['chr'],data['strand'],data['gene_name'],
                      data['transcript_id'],data['pro_id'],data['base1'],
                      data['base2'],data['base3'],data['aa_genome_seq'],
                      data['aa_position'],data['type'],data['reg_from'],
                      data['reg_to'],data['reg_seq'],NGG_pos])

In [17]:
pam_f = DataFrame(pam_f,columns = ['chr', 'strand', 'gene_name', 'transcript_id', 'pro_id',
       'base1', 'base2', 'base3', 'aa_genome_seq', 'aa_position', 'type',
       'reg_from', 'reg_to', 'reg_seq', 'NGG_pos'])

In [18]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400382,18400383,18400384,AGC,127,nosp,18400396,18400402,GTGGCAG,2
1,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728410,75728411,75728412,AGT,14,nosp,75728424,75728430,TCATTGG,5
2,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728482,75728483,75728484,AGT,38,nosp,75728496,75728502,TTCGGTT,3
3,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75733566,75733567,75733568,AGT,109,nosp,75733580,75733586,TTATGGA,4
4,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75761182,75761183,75761184,AGT,336,nosp,75761196,75761202,AGCTTGG,5


In [19]:
pam_f.shape

(118353, 15)

In [20]:
pam_f.loc[:,'distance'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - pam_f['base1']

In [21]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400382,18400383,18400384,AGC,127,nosp,18400396,18400402,GTGGCAG,2,15
1,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728410,75728411,75728412,AGT,14,nosp,75728424,75728430,TCATTGG,5,18
2,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728482,75728483,75728484,AGT,38,nosp,75728496,75728502,TTCGGTT,3,16
3,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75733566,75733567,75733568,AGT,109,nosp,75733580,75733586,TTATGGA,4,17
4,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75761182,75761183,75761184,AGT,336,nosp,75761196,75761202,AGCTTGG,5,18


In [22]:
pam_f.loc[pam_f['distance'].isin([14,15,16]),'sg_from'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - 19
pam_f.loc[pam_f['distance'].isin([14,15,16]),'sg_to'] = pam_f['sg_from'] + 19 - 1

pam_f.loc[pam_f['distance'].isin([13,17]),'sg_from'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - 20
pam_f.loc[pam_f['distance'].isin([13,17]),'sg_to'] = pam_f['sg_from'] + 20 - 1

pam_f.loc[pam_f['distance'].isin([18]),'sg_from'] = pam_f['reg_from'] + pam_f['NGG_pos'] - 1 - 21
pam_f.loc[pam_f['distance'].isin([18]),'sg_to'] = pam_f['sg_from'] + 21 - 1

pam_f['sg_from'] = pam_f['sg_from'].map(lambda x:('%.0f')%x).astype(int)
pam_f['sg_to'] = pam_f['sg_to'].map(lambda x:('%.0f')%x).astype(int)

In [23]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400382,18400383,18400384,AGC,127,nosp,18400396,18400402,GTGGCAG,2,15,18400378,18400396
1,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728410,75728411,75728412,AGT,14,nosp,75728424,75728430,TCATTGG,5,18,75728407,75728427
2,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728482,75728483,75728484,AGT,38,nosp,75728496,75728502,TTCGGTT,3,16,75728479,75728497
3,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75733566,75733567,75733568,AGT,109,nosp,75733580,75733586,TTATGGA,4,17,75733563,75733582
4,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75761182,75761183,75761184,AGT,336,nosp,75761196,75761202,AGCTTGG,5,18,75761179,75761199


In [24]:
pam_f.loc[:,'sg_seq'] = pam_f.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_from'] - 1: x['sg_to']]).upper(),
            axis=1)

In [25]:
pam_f.loc[:,'pam'] = pam_f.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_to'] : x['sg_to'] + 3]).upper(),
            axis=1)

In [26]:
pam_f.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to,sg_seq,pam
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400382,18400383,18400384,AGC,127,nosp,18400396,18400402,GTGGCAG,2,15,18400378,18400396,TGGAAGCTCCTCCCAGATG,TGG
1,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728410,75728411,75728412,AGT,14,nosp,75728424,75728430,TCATTGG,5,18,75728407,75728427,AGAAGTATTTCTCGTTTTCAT,TGG
2,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728482,75728483,75728484,AGT,38,nosp,75728496,75728502,TTCGGTT,3,16,75728479,75728497,TTTAGTTTTGGTATATGTT,CGG
3,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75733566,75733567,75733568,AGT,109,nosp,75733580,75733586,TTATGGA,4,17,75733563,75733582,ATTAGTGAAGAATTGGCTTA,TGG
4,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75761182,75761183,75761184,AGT,336,nosp,75761196,75761202,AGCTTGG,5,18,75761179,75761199,ATGAGTTACCAGAGAGCAGCT,TGG


In [27]:
pam_f.shape

(118353, 20)

In [28]:
pam_f.loc[:,'aa_id'] = pam_f['chr'].str.cat(pam_f['base1'].astype(str),sep='-')
pam_f.loc[:,'sg_id'] = pam_f['aa_id'].str.cat(pam_f['sg_seq'].astype(str),sep='-')

In [29]:
pam_f['aa_id'].unique().shape

(56804,)

In [30]:
pam_f['sg_id'].unique().shape

(77822,)

In [31]:
pam_f['sg_seq'].unique().shape

(76164,)

In [32]:
#reverse strand sgRNA

In [33]:
genpos_r.loc[:,'reg_from'] = genpos_r['base3'] - 20 
genpos_r.loc[:,'reg_to'] = genpos_r['base3'] - 14

In [34]:
genpos_r.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to
253058,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9115820,9115821,9115822,AGT,10,nosp,9115802,9115808
253059,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113474,9113475,9113476,AGC,52,nosp,9113456,9113462
253060,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113447,9113448,9113449,AGT,61,nosp,9113429,9113435
253061,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113414,9113415,9113416,AGC,72,nosp,9113396,9113402
253062,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9112426,9112427,9112428,AGT,127,nosp,9112408,9112414


In [35]:
genpos_r.loc[:,'reg_seq'] = genpos_r.apply(
            lambda x: str(fa[x['chr']].seq[x['reg_from'] - 1: x['reg_to']].reverse_complement()).upper(),
            axis=1)

In [36]:
genpos_r.head()

,chr,feature,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq
253058,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9115820,9115821,9115822,AGT,10,nosp,9115802,9115808,CCTCTTG
253059,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113474,9113475,9113476,AGC,52,nosp,9113456,9113462,GACAGTG
253060,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113447,9113448,9113449,AGT,61,nosp,9113429,9113435,GTCTGTC
253061,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9113414,9113415,9113416,AGC,72,nosp,9113396,9113402,CCTGGAG
253062,chr12,CDS,-,A2M,NM_000014,A2MG_HUMAN,9112426,9112427,9112428,AGT,127,nosp,9112408,9112414,CCAGACA


In [37]:
pam_r = []
for index, data in genpos_r.iterrows():
    for i in re.finditer('(?=GG)',data['reg_seq']):
        NGG_pos = i.span()[1]
        pam_r.append([data['chr'],data['strand'],data['gene_name'],
                      data['transcript_id'],data['pro_id'],data['base1'],
                      data['base2'],data['base3'],data['aa_genome_seq'],
                      data['aa_position'],data['type'],data['reg_from'],
                      data['reg_to'],data['reg_seq'],NGG_pos])

In [38]:
pam_r = DataFrame(pam_r,columns = ['chr', 'strand', 'gene_name', 'transcript_id', 'pro_id',
       'base1', 'base2', 'base3', 'aa_genome_seq', 'aa_position', 'type',
       'reg_from', 'reg_to', 'reg_seq', 'NGG_pos'])

In [39]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113414,9113415,9113416,AGC,72,nosp,9113396,9113402,CCTGGAG,3
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,0
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,1
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,4
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9095572,9095573,9095574,AGT,660,nosp,9095554,9095560,GGATATG,0


In [40]:
pam_r.shape

(110162, 15)

In [41]:
pam_r.loc[:,'distance'] = pam_r['base3'] - (pam_r['reg_to'] - pam_r['NGG_pos'] + 1)

In [42]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113414,9113415,9113416,AGC,72,nosp,9113396,9113402,CCTGGAG,3,16
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,0,13
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,1,14
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,4,17
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9095572,9095573,9095574,AGT,660,nosp,9095554,9095560,GGATATG,0,13


In [43]:
pam_r.loc[pam_r['distance'].isin([14,15,16]),'sg_from'] = pam_r['reg_to'] - pam_r['NGG_pos'] + 1 + 19
pam_r.loc[pam_r['distance'].isin([14,15,16]),'sg_to'] = pam_r['sg_from'] - 19 + 1

pam_r.loc[pam_r['distance'].isin([13,17]),'sg_from'] = pam_r['reg_to'] - pam_r['NGG_pos'] + 1 + 20
pam_r.loc[pam_r['distance'].isin([13,17]),'sg_to'] = pam_r['sg_from'] - 20 + 1

pam_r.loc[pam_r['distance'].isin([18]),'sg_from'] = pam_r['reg_to'] - pam_r['NGG_pos'] + 1 + 21
pam_r.loc[pam_r['distance'].isin([18]),'sg_to'] = pam_r['sg_from'] - 21 + 1

pam_r['sg_from'] = pam_r['sg_from'].map(lambda x:('%.0f')%x).astype(int)
pam_r['sg_to'] = pam_r['sg_to'].map(lambda x:('%.0f')%x).astype(int)

In [44]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113414,9113415,9113416,AGC,72,nosp,9113396,9113402,CCTGGAG,3,16,9113419,9113401
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,0,13,9110009,9109990
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,1,14,9110007,9109989
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,4,17,9110005,9109986
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9095572,9095573,9095574,AGT,660,nosp,9095554,9095560,GGATATG,0,13,9095581,9095562


In [45]:
pam_r.loc[:,'sg_seq'] = pam_r.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_to'] - 1: x['sg_from']].reverse_complement()).upper(),
            axis=1)

In [46]:
pam_r.loc[:,'pam'] = pam_r.apply(
            lambda x: str(fa[x['chr']].seq[x['sg_to'] - 4 : x['sg_to'] - 1].reverse_complement()).upper(),
            axis=1)

In [47]:
pam_r.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,reg_from,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to,sg_seq,pam
0,chr12,-,A2M,NM_000014,A2MG_HUMAN,9113414,9113415,9113416,AGC,72,nosp,9113396,9113402,CCTGGAG,3,16,9113419,9113401,AGGAGCCTCTTCACTGACC,TGG
1,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,0,13,9110009,9109990,ATGGCAGAGTTTCCAGTTAG,AGG
2,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,1,14,9110007,9109989,GGCAGAGTTTCCAGTTAGA,GGG
3,chr12,-,A2M,NM_000014,A2MG_HUMAN,9110000,9110001,9110002,AGT,180,nosp,9109982,9109988,GGGTGGC,4,17,9110005,9109986,CAGAGTTTCCAGTTAGAGGG,TGG
4,chr12,-,A2M,NM_000014,A2MG_HUMAN,9095572,9095573,9095574,AGT,660,nosp,9095554,9095560,GGATATG,0,13,9095581,9095562,AGTATCAAGTACAAATGAAA,AGG


In [48]:
pam_r.shape

(110162, 20)

In [49]:
pam_r.loc[:,'aa_id'] = pam_r['chr'].str.cat(pam_r['base3'].astype(str),sep='-')
pam_r.loc[:,'sg_id'] = pam_r['aa_id'].str.cat(pam_r['sg_seq'].astype(str),sep='-')

In [50]:
pam_r['aa_id'].unique().shape

(55850,)

In [51]:
pam_r['sg_id'].unique().shape

(76041,)

In [52]:
pam_r['sg_seq'].unique().shape

(74433,)

In [53]:
#sgRNA filter with 0.2 < GC content < 0.8 & remove TTTT

In [54]:
sg = pam_f.append(pam_r)

In [55]:
sg.head()

,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,...,reg_to,reg_seq,NGG_pos,distance,sg_from,sg_to,sg_seq,pam,aa_id,sg_id
0,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400382,18400383,18400384,AGC,127,...,18400402,GTGGCAG,2,15,18400378,18400396,TGGAAGCTCCTCCCAGATG,TGG,chr8-18400382,chr8-18400382-TGGAAGCTCCTCCCAGATG
1,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728410,75728411,75728412,AGT,14,...,75728430,TCATTGG,5,18,75728407,75728427,AGAAGTATTTCTCGTTTTCAT,TGG,chr1-75728410,chr1-75728410-AGAAGTATTTCTCGTTTTCAT
2,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75728482,75728483,75728484,AGT,38,...,75728502,TTCGGTT,3,16,75728479,75728497,TTTAGTTTTGGTATATGTT,CGG,chr1-75728482,chr1-75728482-TTTAGTTTTGGTATATGTT
3,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75733566,75733567,75733568,AGT,109,...,75733586,TTATGGA,4,17,75733563,75733582,ATTAGTGAAGAATTGGCTTA,TGG,chr1-75733566,chr1-75733566-ATTAGTGAAGAATTGGCTTA
4,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75761182,75761183,75761184,AGT,336,...,75761202,AGCTTGG,5,18,75761179,75761199,ATGAGTTACCAGAGAGCAGCT,TGG,chr1-75761182,chr1-75761182-ATGAGTTACCAGAGAGCAGCT


In [56]:
sg.loc[:,'GC_content'] = (sg['sg_seq'].str.count('G') + sg['sg_seq'].str.count('C')) / sg['sg_seq'].str.len()

In [57]:
sg_gc = sg[(sg['GC_content'] >= 0.2) & (sg['GC_content'] <= 0.8)]

In [58]:
sg_gc_4t = sg_gc.append(
    sg_gc[sg_gc['sg_seq'].str.contains('TTTT')]).drop_duplicates(keep=False)

In [59]:
sg_gc_4t.columns

Index(['chr', 'strand', 'gene_name', 'transcript_id', 'pro_id', 'base1',
       'base2', 'base3', 'aa_genome_seq', 'aa_position', 'type', 'reg_from',
       'reg_to', 'reg_seq', 'NGG_pos', 'distance', 'sg_from', 'sg_to',
       'sg_seq', 'pam', 'aa_id', 'sg_id', 'GC_content'],
      dtype='object')

In [60]:
sg_gc_4t = sg_gc_4t[['sg_id','aa_id','chr','strand','gene_name','transcript_id',
                    'pro_id','base1','base2','base3','aa_genome_seq','aa_position',
                    'type','sg_seq','sg_from','sg_to','pam','GC_content']]

In [61]:
sg_gc_4t.head()

,sg_id,aa_id,chr,strand,gene_name,transcript_id,pro_id,base1,base2,base3,aa_genome_seq,aa_position,type,sg_seq,sg_from,sg_to,pam,GC_content
0,chr8-18400382-TGGAAGCTCCTCCCAGATG,chr8-18400382,chr8,+,NAT2,NM_000015,ARY2_HUMAN,18400382,18400383,18400384,AGC,127,nosp,TGGAAGCTCCTCCCAGATG,18400378,18400396,TGG,0.578947
3,chr1-75733566-ATTAGTGAAGAATTGGCTTA,chr1-75733566,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75733566,75733567,75733568,AGT,109,nosp,ATTAGTGAAGAATTGGCTTA,75733563,75733582,TGG,0.300000
4,chr1-75761182-ATGAGTTACCAGAGAGCAGCT,chr1-75761182,chr1,+,ACADM,NM_000016,ACADM_HUMAN,75761182,75761183,75761184,AGT,336,nosp,ATGAGTTACCAGAGAGCAGCT,75761179,75761199,TGG,0.476190
7,chr12-120737428-TCACCAGTGGTGACAAAAT,chr12-120737428,chr12,+,ACADS,NM_000017,ACADS_HUMAN,120737428,120737429,120737430,AGT,145,nosp,TCACCAGTGGTGACAAAAT,120737423,120737441,TGG,0.421053
8,chr12-120738409-GACAGCATCCTGGGGGAGCC,chr12-120738409,chr12,+,ACADS,NM_000017,ACADS_HUMAN,120738409,120738410,120738411,AGC,252,nosp,GACAGCATCCTGGGGGAGCC,120738406,120738425,AGG,0.700000


In [62]:
sg_gc_4t['sg_id'].unique().shape

(146365,)

In [63]:
sg_gc_4t['aa_id'].unique().shape

(107757,)

In [64]:
sg_gc_4t['sg_seq'].unique().shape

(142176,)

In [65]:
sg_gc_4t.to_csv('../data/ser/sg_ser.txt',sep='\t',index=False)